In [ ]:
import os
os.chdir("..")

In [ ]:
from unitcellengine.design import UnitcellDesign, UnitcellDesigns
import numpy as np
import pyvista as pv
from tqdm.notebook import tqdm
import plotly.graph_objects as go

# Set numpy precision to allow for better printout of homogenized matrices
np.set_printoptions(precision=4)

# Single point design

Create a single point design of a unit aspect ratio Octet truss lattice and investigate its properties.
Note that, in the default setup, the assumed material has a modulus of elasticity of 1 and Poisson's ratio of 0.3.

In [ ]:
# Define unitcell
design = UnitcellDesign('Octet', 1, 1, 1, thickness=0.1)

# The default behavior is to reuse existing date. Set to False to force regeneration.
reuse = True

# Generate geometry (which calculated propertes like relative denstiy and relative surface area)
design.generateGeometry(reuse=reuse)

# Generate mesh
design.generateMesh(reuse=reuse)

# Calculate homogenized elastic properties
design.homogenizationElastic.run(reuse)

# Post process homogenization results
design.homogenizationElastic.process()

# Calculate homogenized conductance properties
design.homogenizationConductance.run(reuse)

# Post process conductance results
design.homogenizationConductance.process()

# Print the homogenizated stiffness matrix
print(design.homogenizationElastic)
print(design.homogenizationConductance)

## Visualize geometry

The scalar colors corresponds to the underlying signed-distance field, which defines the distance of a point to the closed point on the geometry surface (thus, a value of 0 corresponds to the surface of the geometry).

Note: the python package "trame", along with the trame-jupyter-extension jupyter extension, 
are required for display of an interactive plot. See https://tutorial.pyvista.org/tutorial/00_jupyter/index.html
for more details

In [ ]:
geometry = design.geometry.visualizeVTK()
geometry.plot()

## Visualize anistropy of elastic properties

For more details regaring the below visualizations, see Nordmann, J., Aßmus, M., & Altenbach, H. (2018). *Visualising elastic anisotropy: theoretical background and computational implementation*. Continuum Mechanics and Thermodynamics, 30, 689-708.

In [ ]:
design.homogenizationElastic.plotE()

In [ ]:
design.homogenizationElastic.plotG()

# Export design to a centeralized database

Export to a centeralized HDF5 database

In [ ]:
design.export()

The centralized database is located in the root database folder and can be found as follows:

In [ ]:
design.databaseFilename

# Design groups

Generate the properties for a set of lattices and manage their properties as a group.

In [ ]:
cases = [
    dict(length=1, width=1, height=1, thickness=0.3),
    dict(length=2, width=1, height=1, thickness=0.3),
    dict(length=3, width=1, height=1, thickness=0.3),
    dict(length=4, width=1, height=1, thickness=0.3),
]

# The default behavior is to reuse existing date. Set to False to force regeneration.
reuse = True

for case in tqdm(cases):
    # Define unitcell
    design = UnitcellDesign('Octet', **case)
    
    # Generate geometry (which calculated propertes like relative denstiy and relative surface area)
    design.generateGeometry(reuse=reuse)
    
    # Generate mesh
    design.generateMesh(reuse=reuse)
    
    # Calculate homogenized elastic properties
    design.homogenizationElastic.run(reuse)
    
    # Post process homogenization results
    design.homogenizationElastic.process(reuse=False)

Load the data for all of the Octet truss designs

In [ ]:
designs = UnitcellDesigns("Octet", form="graph")

Get various properties from the set of unitcell designs

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=designs.lengths, y=designs.relativeDensities))
fig.update_xaxes(title="Unitcell length")
fig.update_yaxes(title="Relative density")

Calculate the normalized elastic stiffness in the 1 direction (i.e., length-wise direction)

In [ ]:
E11s = [d.homogenizationElastic.Ei([1, 0, 0]) for d in designs.designs]
fig = go.Figure()
fig.add_trace(go.Scatter(x=designs.relativeDensities, y=E11s))
fig.update_yaxes(title="E<sub>11</sub>")
fig.update_xaxes(title="Relative density")